In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Extracting the fire data set

In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('drive/MyDrive/FIRE_Dataset_EN_2010.rar').extractall('/content/')


     |████████████████████████████████| 81kB 5.0MB/s 


Importing required libraries

In [ ]:
import os
import re
import math
import glob
import numpy as np
from numpy.linalg import norm
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import collections
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
class Preprocessing_Data:
  
  def load_data(self):            #To Load the Data Available for applying preprocessing on it
    global doc                          #here as the data has been extracted ,we can not easily load the data in to the RAM.
    global total_documents
    total_documents = 0
    doc = list()                                     #contains real document id list
    
    data = list()                                      #contains list of list of data for each file .each list within list has one corresponding file data
    files = list()                                   #contains list of all file names in the root directory and its subdirectoris  
    path = "/content/FIRE_Dataset_EN_2010"        #root directory
    for root, dirs, f in os.walk(path):
      for item in f:
        if '.utf8' in item:                         #getting only ".utf8" files 
          files.append(os.path.join(root, item))    
    for name in files:                               #traversing all the ".utf8" files and will be extracting the data from it in the ram 
      #if files.index(name) % 5000 == 0:                      #check if name contains index type files or not
        #print("record loaded = ",files.index(name))
      check = re.findall("index",str(name))
      if len(check) != 0:  #contains index then continue and ignore this file
         continue
      if name == '.config' or name == 'sample_data':
        continue
      else:
        f = open(name,'r')
        soup = BeautifulSoup(f,"html.parser")
        required_data = soup.find("text")
        temp = required_data.text
        check = re.findall("[A-Za-z0-9]",temp)                #check if document contains any words or not
        if len(check) == 0:
          continue
        data.append(temp)       
        doc_id = soup.find("docno")
        doc.append(doc_id.text)
        total_documents += 1
        
    return data

  #function to preprocess the data
  def preprocess(self,data):
      #retrieving each doc data and then removing numerical and punctuation from each document
      stop_words = stopwords.words('english')
      lemmatizer = WordNetLemmatizer()
      preprocessed_data = list()
      for doc_text in data:
        #remooving numerical and punctuation#   
        doc_text = re.sub(r'[^\w\s]', '',doc_text)
        doc_text = re.sub('\d','',doc_text)
        #tokenizing data
        data_tokenized = nltk.word_tokenize(doc_text)
        #converting to lowercase
        data_lowercase = [word.lower() for word in data_tokenized] 
        #removing stopwords
        data_without_stopwords = list()       
        for word in data_lowercase:
          if word not in stop_words:
            data_without_stopwords.append(word)
        #data lemmatization
        prev_data = list() 
        for word in data_without_stopwords:
          prev_data.append(lemmatizer.lemmatize(word,pos='v'))
        preprocessed_data.append(prev_data)
      return preprocessed_data
  
  def extractUnique(self,data):
    #return unique feature from the entire corpus
    return collections.Counter([x for sub_list in data for x in sub_list])

In [ ]:
doc = list()
p1 = Preprocessing_Data()
data = p1.load_data()
preprocessed_data = p1.preprocess(data)

In [ ]:
len(preprocessed_data)   #printing length of preprocessed data

116553

In [ ]:
with open("corpus_text_tokens.txt", "wb") as fp:   #Pickling
    pickle.dump(preprocessed_data, fp)

In [ ]:
with open("corpus_text_tokens.txt", "rb") as fp:   # Unpickling
    preprocessed_data = pickle.load(fp)

In [ ]:
preprocessed_data[0]

['telegraph',
 'calcutta',
 'bengal',
 'biggest',
 'challenge',
 'ever',
 'front',
 'ashis',
 'chakrabarti',
 'bloodbath',
 'nandigram',
 'first',
 'tragedy',
 'leftruled',
 'bengal',
 'two',
 'years',
 'front',
 'come',
 'power',
 'police',
 'fire',
 'forcible',
 'settlers',
 'marichjhanpi',
 'island',
 'sunderbans',
 'two',
 'occasion',
 'kill',
 'halfadozen',
 'people',
 'height',
 'gorkhaland',
 'agitation',
 'people',
 'die',
 'police',
 'fire',
 'mob',
 'kill',
 'four',
 'policemen',
 'raid',
 'kalimpong',
 'police',
 'station',
 'police',
 'bullets',
 'kill',
 'congress',
 'supporters',
 'calcuttas',
 'red',
 'road',
 'party',
 'procession',
 'lead',
 'mamata',
 'banerjee',
 'parent',
 'party',
 'try',
 'march',
 'writers',
 'build',
 'case',
 'police',
 'action',
 'involve',
 'fewer',
 'casualties',
 'wednesdays',
 'tragedy',
 'nandigram',
 'may',
 'prove',
 'lefts',
 'biggest',
 'blunder',
 'far',
 'political',
 'consequence',
 'could',
 'far',
 'serious',
 'front',
 'buddhade

Fetching query details

In [ ]:
queries = list()                               #will contain all the preprocessed queries 
query_ids = list()                             #will contain all the query ids
f = open("/content/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt")
soup = BeautifulSoup(f,"html.parser")
q = soup.find_all("top")

for query_data in q:
  query_ids.append(query_data.num.text)
  queries.append(query_data.desc.text)                            #fetching query(desc) and inserting to query list 
  
#preprocessing the query to get query generator
query_gen = p1.preprocess(queries)

### **CBOW Model**

In [ ]:
from gensim.models import Word2Vec          #importing word2vec for training CBOW and skipgram model

In [ ]:
cbow_model = Word2Vec(preprocessed_data, size=300, window=2, min_count=1, sg=0, workers=3, alpha=0.025)  #CBOW model of window size 2 and size=300

In [ ]:
cbow_model.train(preprocessed_data, total_examples=cbow_model.corpus_count, epochs=4)      #training the model with epochs=4

(113809790, 116052044)

In [ ]:
cbow_model.save('cbow_model.model')              #save the model

For finding the cosine similarity between document and query here I have created two arrays using the data of model.

In [ ]:
document_array = []
query_array = []
for i in preprocessed_data:
  temp = np.zeros((300,), dtype=float)
  for j in i:
      temp = temp + cbow_model[j]
  document_array.append(temp)
print("length of document array:",len(document_array))
#query array
for i in query_gen:
  temp = np.zeros((300,),dtype=float)
  for j in i:
      try:
        temp = temp + cbow_model[j]
      except:
        continue
  query_array.append(temp)
print("length of query array:",len(query_array))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


length of document array: 116553
length of query array: 50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Calculating the cosine similarity of document_array and query_array

In [ ]:
dict_sim = {}
x=[x for x in range(76,126)]
for i in range(len(query_gen)):
  temp=[]
  for j in range(len(preprocessed_data)):
    #sim = cbow_model.similarity(np.ndarray(query_gen[i]).reshape(1,-1),np.ndarray(preprocessed_data[j]).reshape(1,-1))
    sim = cosine_similarity(query_array[i].reshape(1,300),document_array[j].reshape(1,300))
    temp.append(sim)
  key=x[i]
  dict_sim.update({key : temp})

In [ ]:
df = pd.DataFrame.from_dict(dict_sim,orient='index',columns=doc)   #saving the dictionary in dataframe
df

,1070315_bengal_story_7519501.utf8,1070518_bengal_story_7794967.utf8,1070923_bengal_story_8350864.utf8,1070701_bengal_story_7999605.utf8,1070924_bengal_story_8352692.utf8,1070314_bengal_story_7513991.utf8,1070120_bengal_story_7287489.utf8,1070103_bengal_story_7214140.utf8,1070421_bengal_story_7676468.utf8,1070830_bengal_story_8254837.utf8,1070712_bengal_story_8049312.utf8,1070919_bengal_story_8334613.utf8,1070628_bengal_story_7986070.utf8,1070419_bengal_story_7667510.utf8,1070801_bengal_story_8133239.utf8,1070308_bengal_story_7488516.utf8,1070227_bengal_story_7446026.utf8,1070808_bengal_story_8165373.utf8,1070716_bengal_story_8064498.utf8,1070429_bengal_story_7713101.utf8,1070904_bengal_story_8273668.utf8,1070224_bengal_story_7435352.utf8,1070124_bengal_story_7303292.utf8,1070222_bengal_story_7425901.utf8,1070618_bengal_story_7938126.utf8,1070923_bengal_story_8350862.utf8,1070720_bengal_story_8083353.utf8,1070509_bengal_story_7753444.utf8,1070321_bengal_story_7545629.utf8,1070302_bengal_story_7459693.utf8,1070109_bengal_story_7238126.utf8,1070204_bengal_story_7348287.utf8,1070404_bengal_story_7605097.utf8,1070228_bengal_story_7451705.utf8,1070216_bengal_story_7400760.utf8,1070906_bengal_story_8282885.utf8,1070117_bengal_story_7273021.utf8,1070203_bengal_story_7344540.utf8,1070829_bengal_story_8250269.utf8,1070207_bengal_story_7361029.utf8,...,1060614_foreign_story_6350454.utf8,1061129_foreign_story_7065326.utf8,1060427_foreign_story_6151276.utf8,1060530_foreign_story_6287333.utf8,1061220_foreign_story_7162378.utf8,1060112_foreign_story_5710889.utf8,1060204_foreign_story_5803832.utf8,1060629_foreign_story_6413108.utf8,1060121_foreign_story_5748236.utf8,1060729_foreign_story_6541378.utf8,1060323_foreign_story_6003209.utf8,1061119_foreign_story_7022197.utf8,1060102_foreign_story_5670231.utf8,1060213_foreign_story_5839769.utf8,1060629_foreign_story_6413896.utf8,1061006_foreign_story_6833241.utf8,1060319_foreign_story_5984948.utf8,1060822_foreign_story_6640939.utf8,1061012_foreign_story_6859591.utf8,1060923_foreign_story_6783585.utf8,1060207_foreign_story_5814557.utf8,1060527_foreign_story_6276480.utf8,1061026_foreign_story_6917584.utf8,1060919_foreign_story_6764177.utf8,1060717_foreign_story_6488495.utf8,1060912_foreign_story_6733731.utf8,1060421_foreign_story_6126452.utf8,1060205_foreign_story_5807150.utf8,1060503_foreign_story_6176065.utf8,1060224_foreign_story_5889240.utf8,1060617_foreign_story_6364122.utf8,1061006_foreign_story_6831713.utf8,1060521_foreign_story_6250642.utf8,1060424_foreign_story_6137728.utf8,1060724_foreign_story_6517897.utf8,1061211_foreign_story_7124728.utf8,1061212_foreign_story_7126694.utf8,1060716_foreign_story_6485190.utf8,1061202_foreign_story_7080150.utf8,1060124_foreign_story_5759204.utf8
76,[[0.671363205764619]],[[0.6440773677140978]],[[0.44740157970530714]],[[0.6685120444053148]],[[0.683133010149375]],[[0.629927052630283]],[[0.5521128956085187]],[[0.552928282386712]],[[0.5642290746550058]],[[0.4981936719266402]],[[0.6113593433141056]],[[0.6336143622479373]],[[0.6490850774841489]],[[0.6799492028624459]],[[0.6090382382368198]],[[0.5967874395335404]],[[0.5185201335561574]],[[0.6300526747297764]],[[0.5658912059717216]],[[0.5106838085469794]],[[0.6051469136027062]],[[0.6087396413775948]],[[0.5889032775788644]],[[0.6785512796294093]],[[0.4990250644424529]],[[0.5716990136547646]],[[0.5946300404855759]],[[0.6404949514708064]],[[0.6600731564820504]],[[0.6866077095967602]],[[0.6602450757976448]],[[0.5847455002371034]],[[0.5588561655940437]],[[0.5160737428738299]],[[0.4689688569729732]],[[0.5574729585665372]],[[0.548758924542196]],[[0.5638272244830892]],[[0.5908329115803724]],[[0.566065408258818]],...,[[0.6208257268034049]],[[0.48704304827950723]],[[0.5214044164808784]],[[0.5747217867283284]],[[0.6724412521567321]],[[0.46911608955993783]],[[0.6523773335848284]],[[0.572252789738884]],[[0.5607248181232917]],[[0.5081760892261847]],[[0.5753140826115412]],[[0.45855636234045005]],[[0.5268528213407082]],[[0.53796333

Finding the rank list based on cosine similarity dataframe

In [ ]:
final_rank_list={}         
indexlist=df.index.values.tolist()
for index in indexlist:
  sorted_val=df.loc[index,:].sort_values(ascending=False).index.values.tolist()
  value=','.join(sorted_val[0:10])
  final_rank_list.update({index:str(value)})

In [ ]:
final_rank_list

{76: '1050419_bengal_story_4633071.utf8,1060505_bengal_story_6184689.utf8,1070603_nation_story_7869357.utf8,1070611_nation_story_7906812.utf8,1061115_nation_story_7003356.utf8,1060905_nation_story_6699721.utf8,1061203_nation_story_7084990.utf8,1060331_frontpage_story_6038335.utf8,1060517_nation_story_6234124.utf8,1060521_frontpage_story_6250756.utf8',
 77: '1060717_foreign_story_6488684.utf8,1060719_foreign_story_6496938.utf8,1051229_foreign_story_5657775.utf8,1060611_foreign_story_6337687.utf8,1060814_foreign_story_6609200.utf8,1050804_foreign_story_5072554.utf8,1060715_foreign_story_6482002.utf8,1060727_foreign_story_6531510.utf8,1060714_foreign_story_6477224.utf8,1060720_foreign_story_6501311.utf8',
 78: '1050616_nation_story_4874434.utf8,1070918_nation_story_8329881.utf8,1050617_nation_story_4878996.utf8,1041113_nation_story_3999377.utf8,1061009_frontpage_story_6846053a.utf8,1060312_nation_story_5958120.utf8,1050725_nation_story_5030947.utf8,1050701_nation_story_4936920.utf8,104110

In [ ]:
#fetching the document v/s relevance for each query
f = open("/content/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt/en.qrels.76-125.2010.txt","r")
query_doc_relevant = dict()
lines = f.readlines()
for line in lines:
  line_split = line.split(" ")
  line_split[3] =  line_split[3].strip()
  query_no = line_split[0]
  doc_name = line_split[2]
  relevance_score = line_split[3]
  if query_no == " ":
    continue 
  else:
    result = query_doc_relevant.get(query_no,0)
    if result == 0:
      query_doc_relevant[query_no] = [[doc_name,relevance_score]]
    else:
      query_doc_relevant[query_no].append([doc_name,relevance_score]) 

Calculating MAP for CBOW model

In [ ]:
queries_precision = list()
for no in (range(76,126)):
   #query contains [[docname,cosine_similarity],[..],...]
   list_fetched_doc = final_rank_list[int(no)]
   #counting no of correctly retrieved document by filtering all the document which is fetched correctly by system  
   ground_query = query_doc_relevant[str(no)]
   count = 0 
   for ground in ground_query:
     if ground[0] in list_fetched_doc:
        if ground[1] == "1":
          count = count + 1   
   queries_precision.append(float(count)/10) 

#calculating map
map = float(sum(queries_precision)/len(queries_precision))
print("MAP(CBOW):",map)

MAP(CBOW): 0.262


## **Skipgram Model**

In [ ]:
skipgram_model = Word2Vec(preprocessed_data, size=300, window=2, min_count=1, sg=1, workers=3, alpha=0.025)  #skipgram model of window size = 2 and size = 300

In [ ]:
skipgram_model.train(preprocessed_data, total_examples=skipgram_model.corpus_count, epochs=4)   #training the model

(113808281, 116052044)

In [ ]:
skipgram_model.save('skipgram_model.model')           #save the skipgram model

Use the same approach for finding cosine similarity of document and query as in the CBOW model

In [ ]:
document_array_sg = []
query_array_sg = []
for i in preprocessed_data:
  temp = np.zeros((300,), dtype=float)
  for j in i:
      temp = temp + skipgram_model[j]
  document_array_sg.append(temp)
print("length of document array:",len(document_array))
#query array
for i in query_gen:
  temp = np.zeros((300,),dtype=float)
  for j in i:
      try:
        temp = temp + skipgram_model[j]
      except:
        continue
  query_array_sg.append(temp)
print("length of query array:",len(query_array_sg))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


length of document array: 116553
length of query array: 50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
dict_sim_sg = {}
x=[x for x in range(76,126)]
for i in range(len(query_gen)):
  temp=[]
  for j in range(len(preprocessed_data)):
    sim = cosine_similarity(query_array_sg[i].reshape(1,300),document_array_sg[j].reshape(1,300))
    temp.append(sim)
  key=x[i]
  dict_sim_sg.update({key : temp})

In [ ]:
df_sg = pd.DataFrame.from_dict(dict_sim_sg, orient='index', columns=doc) 
df_sg.head(5)

,1070315_bengal_story_7519501.utf8,1070518_bengal_story_7794967.utf8,1070923_bengal_story_8350864.utf8,1070701_bengal_story_7999605.utf8,1070924_bengal_story_8352692.utf8,1070314_bengal_story_7513991.utf8,1070120_bengal_story_7287489.utf8,1070103_bengal_story_7214140.utf8,1070421_bengal_story_7676468.utf8,1070830_bengal_story_8254837.utf8,1070712_bengal_story_8049312.utf8,1070919_bengal_story_8334613.utf8,1070628_bengal_story_7986070.utf8,1070419_bengal_story_7667510.utf8,1070801_bengal_story_8133239.utf8,1070308_bengal_story_7488516.utf8,1070227_bengal_story_7446026.utf8,1070808_bengal_story_8165373.utf8,1070716_bengal_story_8064498.utf8,1070429_bengal_story_7713101.utf8,1070904_bengal_story_8273668.utf8,1070224_bengal_story_7435352.utf8,1070124_bengal_story_7303292.utf8,1070222_bengal_story_7425901.utf8,1070618_bengal_story_7938126.utf8,1070923_bengal_story_8350862.utf8,1070720_bengal_story_8083353.utf8,1070509_bengal_story_7753444.utf8,1070321_bengal_story_7545629.utf8,1070302_bengal_story_7459693.utf8,1070109_bengal_story_7238126.utf8,1070204_bengal_story_7348287.utf8,1070404_bengal_story_7605097.utf8,1070228_bengal_story_7451705.utf8,1070216_bengal_story_7400760.utf8,1070906_bengal_story_8282885.utf8,1070117_bengal_story_7273021.utf8,1070203_bengal_story_7344540.utf8,1070829_bengal_story_8250269.utf8,1070207_bengal_story_7361029.utf8,...,1060614_foreign_story_6350454.utf8,1061129_foreign_story_7065326.utf8,1060427_foreign_story_6151276.utf8,1060530_foreign_story_6287333.utf8,1061220_foreign_story_7162378.utf8,1060112_foreign_story_5710889.utf8,1060204_foreign_story_5803832.utf8,1060629_foreign_story_6413108.utf8,1060121_foreign_story_5748236.utf8,1060729_foreign_story_6541378.utf8,1060323_foreign_story_6003209.utf8,1061119_foreign_story_7022197.utf8,1060102_foreign_story_5670231.utf8,1060213_foreign_story_5839769.utf8,1060629_foreign_story_6413896.utf8,1061006_foreign_story_6833241.utf8,1060319_foreign_story_5984948.utf8,1060822_foreign_story_6640939.utf8,1061012_foreign_story_6859591.utf8,1060923_foreign_story_6783585.utf8,1060207_foreign_story_5814557.utf8,1060527_foreign_story_6276480.utf8,1061026_foreign_story_6917584.utf8,1060919_foreign_story_6764177.utf8,1060717_foreign_story_6488495.utf8,1060912_foreign_story_6733731.utf8,1060421_foreign_story_6126452.utf8,1060205_foreign_story_5807150.utf8,1060503_foreign_story_6176065.utf8,1060224_foreign_story_5889240.utf8,1060617_foreign_story_6364122.utf8,1061006_foreign_story_6831713.utf8,1060521_foreign_story_6250642.utf8,1060424_foreign_story_6137728.utf8,1060724_foreign_story_6517897.utf8,1061211_foreign_story_7124728.utf8,1061212_foreign_story_7126694.utf8,1060716_foreign_story_6485190.utf8,1061202_foreign_story_7080150.utf8,1060124_foreign_story_5759204.utf8
76,[[0.8104276249161434]],[[0.7887704287971941]],[[0.6909965106657572]],[[0.8018896354274225]],[[0.8068539255084614]],[[0.785066165545091]],[[0.7436195851791019]],[[0.7592974386534618]],[[0.7424938595624756]],[[0.6949972088479727]],[[0.7727282372001544]],[[0.7951814490438994]],[[0.8004853906222467]],[[0.8078108831669583]],[[0.7659554384083804]],[[0.7621417120644991]],[[0.7340711117190624]],[[0.7900109731144074]],[[0.7430349055312747]],[[0.7120030177255654]],[[0.7790487682358128]],[[0.7869609057695689]],[[0.7712265816100684]],[[0.805086069180566]],[[0.7270147786280787]],[[0.7667818460269038]],[[0.777749407362163]],[[0.7995226349556118]],[[0.7916001007049628]],[[0.8112737740684915]],[[0.8017822575913658]],[[0.7583727762251403]],[[0.7528194435216542]],[[0.7476684947399801]],[[0.7072737879396265]],[[0.7537527710410368]],[[0.7545650199890687]],[[0.7290332089417004]],[[0.7624950996299115]],[[0.7622490920721448]],...,[[0.7750433222570743]],[[0.7196234450661173]],[[0.7242421457152178]],[[0.7575771890285211]],[[0.8019203252152343]],[[0.7040938639000538]],[[0.7825315393186663]],[[0.7516759989493972]],[[0.7330953860411624]],[[0.7270717643409934]],[[0.7420040940944869]],[[0.7248574653199776]],[[0.7336898166987074]],[[0.74679943

In [ ]:
final_rank_list_sg={}         
indexlist=df.index.values.tolist()
for index in indexlist:
  sorted_val=df_sg.loc[index,:].sort_values(ascending=False).index.values.tolist()
  value=','.join(sorted_val[0:10])
  final_rank_list_sg.update({index:str(value)})

Calculating MAP for skipgram model

In [ ]:
queries_precision_sg = list()
for no in (range(76,126)):
   #query contains [[docname,cosine_similarity],[..],...]
   list_fetched_doc = final_rank_list_sg[int(no)]
   #counting no of correctly retrieved document by filtering all the document which is fetched correctly by system  
   ground_query = query_doc_relevant[str(no)]
   count = 0 
   for ground in ground_query:
     if ground[0] in list_fetched_doc:
        if ground[1] == "1":
          count = count + 1   
   queries_precision_sg.append(float(count)/10) 

#calculating map
map = float(sum(queries_precision_sg)/len(queries_precision_sg))
print("MAP(Skipgram):",map)

MAP: 0.27999999999999997
